In [1]:
from array import array
import reprlib
import math


### N-wymiarowy Vector

In [2]:
class Vector:
    typecode = "d"

    def __init__(self, components):
        # Chroniony atrybut self._components będzie przechowywał
        # tablicę typu array składników wektora.
        self._components = array(self.typecode, components)

    def __iter__(self):
        # Po prostu zwracamy iterator.
        return iter(self._components)

    def __repr__(self):
        # Używamy reprlib.repr() w celu zredukowania pod względem
        # długości reprezentacji wektora.
        components = reprlib.repr(self._components)
        # Same składniki.
        components = components[components.find("[") : -1]
        return "Vector({})".format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return bytes([ord(self.typecode)]) + bytes(self._components)

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __bool__(self):
        return bool(abs(self))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)


In [3]:
v = Vector(range(10))


In [4]:
list(v)


[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]

In [5]:
abs(v)


16.881943016134134

In [6]:
bool(v)


True

In [7]:
bytes(v)


b'd\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\x18@\x00\x00\x00\x00\x00\x00\x1c@\x00\x00\x00\x00\x00\x00 @\x00\x00\x00\x00\x00\x00"@'

In [8]:
repr(v)


'Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])'

In [9]:
str(v)


'(0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0)'

In [10]:
Vector.frombytes(bytes(v))


Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])

### Protokoły i kacze typowanie

W kontekście programowania obiektowego protokół to nieformalny interfejs, definiowany 
tylko w dokumentacji a nie w kodzie. Przykładowo protokół sekwencyjny w Pythonie
wymaga jedynie metod `__len__` i `__getitem__`. Dowolna klasa lub podklasa która implementuje 
te dwie metody może być uzywana wszędzie gdzie jest oczekiwana sekwencja. To czy klasa
jest podklasą tego czy owego nie ma żadnego znaczenia.

Mówimy, że coś jest sekwencją bo działa jak sekwencja i właśnie to ma znaczenie. Nazywamy to kaczym typowaniem.
Nie sprawdzaj czy to jest kaczką. Jeśli cos kwacze jak kaczka i chodzi jak kaczka to jest to kaczka.

In [11]:
import collections

Card = collections.namedtuple("Card", ("rank", "suit"))


# Przykład sekwencji.
class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + ["J", "Q", "K", "A"]
    suits = ["spades", "diamonds", "clubs", "hearts"]

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, index):
        return self._cards[index]


In [12]:
fd = FrenchDeck()
len(fd)


52

In [13]:
fd[24]


Card(rank='K', suit='diamonds')

### Vector - sekwencja z możliwością wycinania

In [14]:
class Vector:
    typecode = "d"

    def __init__(self, components):
        self._components = array(self.typecode, components)

    # Obsługę sekwencji zapewniają dwie proste metody jedno-wierszowe
    # __len__ oraz __getitem__
    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        return self._components[index]

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find("[") : -1]
        return "Vector({})".format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return bytes([ord(self.typecode)]) + bytes(self._components)

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __bool__(self):
        return bool(abs(self))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)


In [15]:
v = Vector(range(10))
v[0]


0.0

In [16]:
v[1], v[-1]


(1.0, 9.0)

In [17]:
# Nie działa to tak jak powinno ponieważ otrzymujemy array.
# Normalnie obiekty pythonowe zwróciłyby nowy obiekt tej samej klasy.
v[1:4]


array('d', [1.0, 2.0, 3.0])

### Jak działa wycinanie

In [18]:
class MySeq:
    def __getitem__(self, index):
        return index


In [19]:
s = MySeq()
s[1]


1

In [20]:
s[1:4]


slice(1, 4, None)

In [21]:
s[1:4:2]


slice(1, 4, 2)

In [22]:
s[1:4:2, 9]


(slice(1, 4, 2), 9)

In [23]:
s[1:4:2, 7:9]


(slice(1, 4, 2), slice(7, 9, None))

In [24]:
dir(slice)


['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'indices',
 'start',
 'step',
 'stop']

In [26]:
help(slice.indices)


Help on method_descriptor:

indices(...)
    S.indices(len) -> (start, stop, stride)
    
    Assuming a sequence of length len, calculate the start and stop
    indices, and the stride length of the extended slice described by
    S. Out of bounds indices are clipped in a manner consistent with the
    handling of normal slices.



Slice to typ wbudowany. Podczas inspekcji znajdujemy mało znaną metodę `indices`. Zakładając, że sekwencja ma długość `len`, metoda `indices` oblicza indeksy `start` i `stop` oraz długość `stride` (krok) wycinka opisywanego jako `S`. Indeksy wykraczające poza granice są przycinane w sposób spójny z obsługą normalnych wycinków. Inaczej mówiąc, `indices` eksponuje logikę zaimplememntowaną we wbudowanych sekwencjach aby z gracją obsługiwać ujemne lub brakujące indeksy oraz wycinki, które są dłuższe niż sekwencja docelowa.

Metoda `slices` wytwarza znormalizowane krotki nieujemnych liczb całkowitych `start`, `stop` i `stride` dopasowanych tak aby pasowały do granic sekwencji zadanej długości.


In [30]:
s = "ABCDE"
s[:10:2]


'ACE'

In [32]:
s[0:5:2]


'ACE'

In [33]:
s[-2:]


'DE'

In [35]:
s[3:5:1]


'DE'

### Vector - świadomy wycinania

In [38]:
import numbers


class Vector:
    typecode = "d"

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __len__(self):
        return len(self._components)

    # Sekwencja świadoma wycinania.
    def __getitem__(self, index):
        # return self._components[index]
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        if isinstance(index, numbers.Integral):
            return self._components[index]
        msg = "{cls.__name__} indices must be integers"
        raise TypeError(msg.format(cls=cls))

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find("[") : -1]
        return "Vector({})".format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return bytes([ord(self.typecode)]) + bytes(self._components)

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __bool__(self):
        return bool(abs(self))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)


In [39]:
v = Vector(range(10))
v


Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])

In [40]:
v[1:4]


Vector([1.0, 2.0, 3.0])

In [42]:
v[-8::2]


Vector([2.0, 4.0, 6.0, 8.0])

In [43]:
v[1:9:3]


Vector([1.0, 4.0, 7.0])

In [44]:
v[1]


1.0

### Dynamiczny dostęp do atrybutów - metoda `__getattr__`

Metoda `__getattr__` jest wywoływana przez interpreter, gdy wyszukiwanie atrybutu kończy się niepowodzeniem. Po otrzymaniu wyrażenia `my_obj.x` Python sprawdza czy instancja `my_obj` ma atrybut o nazwie `x`. Jeśli nie, szukanie przechodzi do klasy `my_obj.__class__` a następnie w głąb grafu dziedziczenia. Jeśli atrybut `x` nie został znaleziony, metoda `__getattr__` definiowana w klasie `my_obj` jest wywoływana z parametrem self i nazwą atrybutu jako łańcuchem `'x'`.

In [48]:
class Vector:
    # Ustawiamy __match_args__ aby umożliwić dynamiczne dopasowywanie wzorców
    # dla dynamicznych atrybutów obsługiwanych przez __getattr__
    __match_args__ = ("x", "y", "z", "t")
    typecode = "d"

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __getattr__(self, name):
        cls = type(self)
        try:
            pos = cls.__match_args__.index(name)
        except ValueError:
            pos = -1
        if 0 <= pos < len(self._components):
            return self._components[pos]
        msg = f"{cls.__name__!r} object has no attribute {name!r}"
        raise AttributeError(msg)

    def __len__(self):
        return len(self._components)

    # Sekwencja świadoma wycinania.
    def __getitem__(self, index):
        # return self._components[index]
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        if isinstance(index, numbers.Integral):
            return self._components[index]
        msg = "{cls.__name__} indices must be integers"
        raise TypeError(msg.format(cls=cls))

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find("[") : -1]
        return "Vector({})".format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return bytes([ord(self.typecode)]) + bytes(self._components)

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __bool__(self):
        return bool(abs(self))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)


In [49]:
v = Vector(range(5))
v


Vector([0.0, 1.0, 2.0, 3.0, 4.0])

In [50]:
v.x  # Faktycznie działa.


0.0

In [51]:
v.x = 10  # To powinno zgłosić wyjątek ale tak się nie dzieje.
v.x


10

In [53]:
v  # Składniki wektora się nie zmieniły!


Vector([0.0, 1.0, 2.0, 3.0, 4.0])

Niespójność powstaje z powodu działania `__getattr__`. Python wywołuje tę metodę jedynie w ostateczności, gdy obiekt nie ma takiego nazwanego atrybutu.
Jednak po przypisaniu `v.x = 10`, obiekt `v` ma atrybut `x`, więc `__getattr__` nie będzie już pozyskiwać `v.x`. Interpreter po prostu zwróci wartość 10 związaną z `v.x`.

In [56]:
class Vector:
    # Ustawiamy __match_args__ aby umożliwić dynamiczne dopasowywanie wzorców
    # dla dynamicznych atrybutów obsługiwanych przez __getattr__
    __match_args__ = ("x", "y", "z", "t")
    typecode = "d"

    def __init__(self, components):
        self._components = array(self.typecode, components)

    # Kodujemy metodę `__setattr__` aby uniknąc niespójnego działania obiektu.
    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1:
            if name in cls.__match_args__:
                error = "read only attribute {attr_name!r}"
            elif name.islower():
                error = "can't set attributes 'a' to 'z' in {cls_name!r}"
            else:
                error = ""
            if error:
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        super().__setattr__(name, value)

    def __getattr__(self, name):
        cls = type(self)
        try:
            pos = cls.__match_args__.index(name)
        except ValueError:
            pos = -1
        if 0 <= pos < len(self._components):
            return self._components[pos]
        msg = f"{cls.__name__!r} object has no attribute {name!r}"
        raise AttributeError(msg)

    def __len__(self):
        return len(self._components)

    # Sekwencja świadoma wycinania.
    def __getitem__(self, index):
        # return self._components[index]
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        if isinstance(index, numbers.Integral):
            return self._components[index]
        msg = "{cls.__name__} indices must be integers"
        raise TypeError(msg.format(cls=cls))

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find("[") : -1]
        return "Vector({})".format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return bytes([ord(self.typecode)]) + bytes(self._components)

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __bool__(self):
        return bool(abs(self))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)


In [57]:
v = Vector(range(5))
v


Vector([0.0, 1.0, 2.0, 3.0, 4.0])

In [58]:
v.x


0.0

In [59]:
v.x = 10


AttributeError: read only attribute 'x'

In [62]:
v.x


0.0

### Haszowalny Vector i szybsze ==

In [63]:
import functools
import operator


In [64]:
functools.reduce(lambda a, b: a * b, range(1, 6))


120

In [100]:
class Vector:
    typecode = "d"

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __eq__(self, other):
        # if not len(self) == len(other):
        #     return False
        # for a, b in zip(self, other):
        #     if not a == b:
        #         return False
        # return True
        return len(self) == len(other) and all(a == b for a, b in zip(self, other))

    def __hash__(self):
        # hashes = (hash(x) for x in self._components)
        hashes = map(hash, self._components)  # Lazy (generator)
        return functools.reduce(operator.xor, hashes, 0)

    def __len__(self):
        return len(self._components)

    def __iter__(self):
        return iter(self._components)


In [99]:
v = Vector(range(1234))
hash(v)


1

### Finalna wersja Vector - formatownie

In [104]:
from array import array
import reprlib
import math
import functools
import operator
import itertools


class Vector:
    __match_args__ = ("x", "y", "z", "t")
    typecode = "d"

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find("[") : -1]
        return f"Vector({components})"

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return bytes([ord(self.typecode)]) + bytes(self._components)

    def __eq__(self, other):
        return len(self) == len(other) and all(a == b for a, b in zip(self, other))

    def __hash__(self):
        hashes = map(hash, self._components)
        return functools.reduce(operator.xor, hashes, 0)

    def __abs__(self):
        return math.hypot(*self)

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        if isinstance(index, numbers.Integral):
            return self._components[index]
        msg = f"{cls.__name__!r} indices must be integers"
        raise TypeError(msg)

    def __getattr__(self, name):
        cls = type(self)
        try:
            pos = cls.__match_args__.index(name)
        except ValueError:
            pos = -1
        if 0 <= pos < len(self._components):
            return self._components[pos]
        msg = f"{cls.__name__!r} object has no attribute {name!r}"
        raise AttributeError(msg)

    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1:
            if name in cls.__match_args__:
                error = "read only attribute {attr_name!r}"
            elif name.islower():
                error = "can't set attributes 'a' to 'z' in {cls_name!r}"
            else:
                error = ""
            if error:
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        super().__setattr__(name, value)

    def angle(self, n):
        r = math.hypot(*self[n:])
        a = math.atan2(r, self[n - 1])
        if n == len(self) - 1 and self[-1] < 0:
            return math.pi * 2 - a
        return a

    def angles(self):
        return (self.angle(n) for n in range(1, len(self)))

    def __format__(self, fmt_spec=""):
        if fmt_spec.endswith("h"):
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)], self.angles())
            outer_fmt = "<{}>"
        else:
            coords = self
            outer_fmt = "({})"
        components = (format(component, fmt_spec) for component in coords)
        return outer_fmt.format(", ".join(components))

    @classmethod
    def frombytes(cls, octects):
        typecode = octects[0]
        memv = memoryview(octects[1:]).cast(typecode)
        return cls(memv)


In [106]:
v = Vector(range(10))
v

Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])

In [110]:
format(v, ".5fh")


'<16.88194, 1.57080, 1.51153, 1.45184, 1.39054, 1.32536, 1.25233, 1.16404, 1.04423, 0.84415>'

In [111]:
format(v, ".1f")

'(0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0)'